# Address missing data

In [1]:
import urllib
import urllib.request
import urllib.parse
import urllib.error
import requests
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd
from pathlib import Path
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from functools import reduce
from start import data_path
import extract_laws
import clean_documents
import numpy as np

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.23) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may in

In [3]:
docs_df = pd.read_csv(os.path.join(data_path,'doi_exemptions_scraped.csv'))
len(docs_df)

824

In [38]:
docs_df[docs_df.title == "South San Antonio ISD"]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,title,level,type,link,text,p_innovation,possible_laws
130,130,614,614,614,640,South San Antonio ISD,Second,pdf,https://www.southsanisd.net//cms/lib/TX0191831...,South San Antonio Independent School District ...,0.999955,"[25.0811, 21.003, 25.112, 25.1113, 21.051, 25...."


In [39]:
extract_laws.get_laws(docs_df[docs_df.title == 'Robstown ISD']['text'].values[0])

[]

## Check TEA website for new districts of innovation

In [40]:
add_new_dois = False

In [41]:
if add_new_dois:
    url = "https://tea.texas.gov/Texas_Schools/District_Initiatives/Districts_of_Innovation/"
    webcontent = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(webcontent, 'html.parser')
    links = soup.find_all('a')
    districts_list = [i for i in map(lambda x: x.get('title'), links) 
                      if i is not None and 'ISD' in i]
    districts_df = pd.DataFrame(districts_list, columns=['title'])
    len(districts_df)
    # Make a new DataFrame from a right join on dois with list of districts
    combined_df = docs_df.merge(districts_df, how='right', left_on='title', right_on='title')

    # Save the district name and link for those districts that we don't have laws for
    missing_districts = combined_df[(combined_df['possible_laws'] == '[]')][['title', 'link']]

    print("Number of missing districts: ", len(missing_districts))
    missing_districts.head()

In [42]:
if add_new_dois == False: 
    # Save the district name and link for those districts that we don't have laws for
    missing_districts = docs_df[(docs_df['possible_laws'] == '[]')][['title', 'link']]
    print("Number of missing districts: ", len(missing_districts))
    missing_districts.head()

Number of missing districts:  274


In [43]:
missing_districts.head()

,title,link
3,Zapata County ISD,https://1.cdn.edl.io/VZ6fi1M6pn8Qk8tS3GetGewCD...
5,Yoakum ISD,http://www.yoakumisd.net/cms/lib3/TX01001553/C...
7,Wylie ISD (221912),http://www.wyliebulldogs.org/cms/One.aspx?port...
10,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...
11,Woodson ISD,https://sites.google.com/a/woodsonisd.net/wood...


## Address districts that are missing due to incorrect link.

### Where possible, incorporate previously (manually) collected links before editing

In [44]:
missing_districts = missing_districts.rename({'link': 'link_scraped'}, axis = 'columns')

# Import previous link fixes
missing_links_old = pd.read_csv(os.path.join(data_path, 'missing_links_corrected.csv'))
missing_links_old = missing_links_old.rename({'link': 'link_manual'}, axis = 'columns')

# merge old and new
missing_links = missing_districts.merge(missing_links_old, how = 'left', left_on = 'title', right_on = 'title')
print("Number of missing links", len(missing_links))

# If I haven't manually fixed the link before, show the link with the highest p(doi)
missing_links['link'] = missing_links.link_manual
missing_links['link'] = missing_links['link'].fillna(missing_links['link_scraped'])
missing_links.tail(20)

Number of missing links 274


,title,link_scraped,Unnamed: 0,link_manual,link
254,Blue Ridge ISD,http://brisd.net/domain/355,256.0,https://docs.google.com/document/d/1jJw9_M2mC_...,https://docs.google.com/document/d/1jJw9_M2mC_...
255,Bland ISD,https://drive.google.com/file/d/0B0CSbOD5Zf1md...,257.0,https://drive.google.com/file/d/0B0CSbOD5Zf1md...,https://drive.google.com/file/d/0B0CSbOD5Zf1md...
256,Blackwell CISD,https://1.cdn.edl.io/b6KPkAODsZKMJ39nycXFAs2Gj...,258.0,https://1.cdn.edl.io/b6KPkAODsZKMJ39nycXFAs2Gj...,https://1.cdn.edl.io/b6KPkAODsZKMJ39nycXFAs2Gj...
257,Benjamin ISD,https://benjamin-isd.net/wp-content/uploads/20...,259.0,http://www.benjaminisd.net/vimages/shared/vnew...,http://www.benjaminisd.net/vimages/shared/vnew...
258,Benavides ISD,http://www.benavidesisd.net/Content2/Innovation,260.0,http://www.benavidesisd.net/userfiles/56/my%20...,http://www.benavidesisd.net/userfiles/56/my%20...
259,Belton ISD,http://www.bisd.net/cms/lib02/TX01001322/Centr...,261.0,http://www.bisd.net/cms/lib02/TX01001322/Centr...,http://www.bisd.net/cms/lib02/TX01001322/Centr...
260,Beeville ISD,https://s3.amazonaws.com/scschoolfiles/380/bis...,262.0,https://s3.amazonaws.com/scschoolfiles/380/bis...,https://s3.amazonaws.com/scschoolfiles/380/bis...
261,Beckville ISD,https://drive.google.com/embeddedfolderview?id...,263.0,https://drive.google.com/file/d/1bosBi0QjvukFz...,https://drive.google.com/file/d/1bosBi0QjvukFz...
262,Ballinger ISD,https://4.files.edl.io/0596/11/05/18/185313-05...,264.0,https://4.files.edl.io/0596/11/05/18/185313-05...,https://4.files.edl.io/0596/11/05/18/185313-05...
263,Austin ISD,https://www.austinisd.org/sites/default/files/...,265.0,https://www.austinisd.org/sites/default/files/...,https://www.austinisd.org/sites/default/files/...


In [45]:
missing_links = missing_links[['title', 'link']]
missing_links.to_csv(os.path.join(data_path, 'missing_links.csv'))

### Manually edit links in missing_links.csv where wrong. Save as missing_links_corrected.csv. Then upload.

In [46]:
missing_links = pd.read_csv(os.path.join(data_path, 'missing_links_corrected.csv'))
missing_links.tail(25)

,Unnamed: 0,title,link
251,251,Brownwood ISD,https://tx02000872.schoolwires.net//cms/lib/TX...
252,252,Brownfield ISD,https://s3.amazonaws.com/scschoolfiles/1457/fi...
253,253,Bronte ISD,http://www.bronteisd.net/storage/UserFileFolde...
254,254,Brazosport ISD,https://www.brazosportisd.net/UserFiles/Server...
255,255,Bowie ISD,http://bowie.esc11.net//cms/lib6/TX02218883/Ce...
256,256,Blue Ridge ISD,https://docs.google.com/document/d/1jJw9_M2mC_...
257,257,Bland ISD,https://drive.google.com/file/d/0B0CSbOD5Zf1md...
258,258,Blackwell CISD,https://1.cdn.edl.io/b6KPkAODsZKMJ39nycXFAs2Gj...
259,259,Benjamin ISD,http://www.benjaminisd.net/vimages/shared/vnew...
260,260,Benavides ISD,http://www.benavidesisd.net/userfiles/56/my%20...


### Extract text from missing_links

In [47]:
texts = []
for link in missing_links.link:
    text = clean_documents.get_plain_text(link)
    texts.append(text)

2019-07-08 09:28:34,479 [MainThread  ] [WARNI]  Tika server returned status: 422


error: HTTP Error 404: Not Found link: http://www.windthorstisd.net/Uploads/50/misc/f283384.pdf


2019-07-08 09:28:52,509 [MainThread  ] [WARNI]  Tika server returned status: 422


Google Error: 'd' is not in listle.com/drive/folders/0B-ALJgmFKVDFZmxBb09rRE9aQ1UiewA_iIXKU5g/edit0Plan.pdff%20Innovation/Innovation%20Plan%20House%20Bill%201842.pdf
Google Error: 'd' is not in listle.com/open?id=1kC4Rn_OgoK8K-s3kvUh4xBPTluPJdtpKtes/News/Documents/FINAL_DOI_PDF_FIle.pdf%20Plan_2017-2022_Adopted%2005.09.2017_Revised%2006.2018.docx


2019-07-08 09:32:46,064 [MainThread  ] [WARNI]  Tika server returned status: 422


error:  link: https://www.eisd.net/site/handlers/filedownload.ashx?moduleinstanceid=37970&dataid=50425&FileName=EDGEWOOD%20ISD%20Disrtict%20of%20Innovation%2010-05-17%20FINAL.pdf
Google Error: 'd' is not in listle.com/open?id=1laYeq1xQHm2ZlQxMjVbDe2V2wdcCm-qFvP75DNtJO58
Google Error: 'd' is not in listle.com/drive/folders/0B3zKmpprcZY1OHRBcEw1d2VFcFE2E.pdfr%20Colmesneil%20ISD%20%281%29.pdff%20School%20DistrictofInnovationFINAL2.14.18.pdf


In [48]:
missing_links['text'] = texts
missing_links['text'] = missing_links['text'].fillna('No text')
len(missing_links)

276

In [49]:
missing_links = clean_documents.remove_whitespace(missing_links, text_col = 'text')
missing_links.tail(25)

,Unnamed: 0,title,link,text
251,251,Brownwood ISD,https://tx02000872.schoolwires.net//cms/lib/TX...,BROWNWOOD INDEPENDENT SCHOOL DISTRICT 2707 Sou...
252,252,Brownfield ISD,https://s3.amazonaws.com/scschoolfiles/1457/fi...,No text
253,253,Bronte ISD,http://www.bronteisd.net/storage/UserFileFolde...,No text
254,254,Brazosport ISD,https://www.brazosportisd.net/UserFiles/Server...,Brazosport Independent School District Local I...
255,255,Bowie ISD,http://bowie.esc11.net//cms/lib6/TX02218883/Ce...,No text
256,256,Blue Ridge ISD,https://docs.google.com/document/d/1jJw9_M2mC_...,Blue Ridge ISD Innovation Plan 2016-2021 HB 18...
257,257,Bland ISD,https://drive.google.com/file/d/0B0CSbOD5Zf1md...,1 | P a g e Introduction The 84th legislature ...
258,258,Blackwell CISD,https://1.cdn.edl.io/b6KPkAODsZKMJ39nycXFAs2Gj...,SCHOOL START DATE (EB LEGAL) (Texas Education ...
259,259,Benjamin ISD,http://www.benjaminisd.net/vimages/shared/vnew...,Page not found – Benjamin ISD About Us Welcome...
260,260,Benavides ISD,http://www.benavidesisd.net/userfiles/56/my%20...,No text


### Extract laws from missing_links

In [50]:
missing_links['possible_laws'] = missing_links.text.apply(extract_laws.get_laws)

In [51]:
print('There are', len(missing_links[missing_links.text == 'No text']), 'documents without text (photocopies, most likely)')
print('There are', len(missing_links[missing_links.text == 'UNAVAILABLE']), 'documents which we cannot access')
print('An additional', len(missing_links[missing_links['possible_laws'].astype(str) == '[]'][(missing_links.text != 'No text') & (missing_links.text != 'UNAVAILABLE')]), 'documents have text but do not contain laws.')

There are 58 documents without text (photocopies, most likely)
There are 2 documents which we cannot access
An additional 89 documents have text but do not contain laws.


/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


### Save now correct observations to 'missing_links_corrected'. Save districts that are still incorrect to missing_text.

In [52]:
missing_links_corrected = missing_links[missing_links['possible_laws'].astype(str) != '[]']
pd.options.display.max_rows = 200
missing_links_corrected

,Unnamed: 0,title,link,text,possible_laws
0,0,Zapata County ISD,https://1.cdn.edl.io/VZ6fi1M6pn8Qk8tS3GetGewCD...,Microsoft Word - Document1 ZAPATA COUNTY INDEP...,"[51.403, 39.306, 39.362]"
1,1,Yoakum ISD,http://www.yoakumisd.net/cms/lib3/TX01001553/C...,DISTRICT OF INNOVATION DISTRICT OF INNOVATION ...,"[21.003, 21.051, 21.102, 21.053, 25.0811, 21.0..."
2,2,Wylie ISD (221912),http://www.wyliebulldogs.org/UserFiles/Servers...,District of Innovation Introduction HB 1842 wa...,"[21.102, 21.003, 21.057, 21.053, 25.113, 25.08..."
4,4,Woodson ISD,https://drive.google.com/file/d/0B5Y_iCwf_UMuT...,Woodson Independent School District District o...,"[25.0811, 25.092, 28.0214, 28.0216, 21.003, 21..."
5,5,Woden ISD,https://drive.google.com/file/d/1j-lifioD9y4Z7...,DISTRICT OF INNOVATION PLAN Introduction The 8...,"[21.053, 21.003, 21.055, 21.401, 25.0811, 21.1..."
8,8,Wimberley ISD,https://1.cdn.edl.io/T7d1mQOc4cS9WyrnOvq4fm2VI...,Wimberley Independent School District District...,"[37.0012, 25.001, 21.102, 25.036, 25.038, 25.0..."
12,12,Whitewright ISD,http://s3.amazonaws.com/scschoolfiles/358/doi_...,WHITEWRIGHT ISD DISTRICT OF INNOVATION School ...,"[25.0811, 21.102]"
13,13,Whitesboro ISD,http://www.whitesboroisd.org/pdf/DOI_Plan_Adop...,WISD District of Innovation Plan 2017-2022 The...,"[25.0811, 21.401, 25.084]"
16,16,Wellman-Union ISD,http://wellman.esc17.net/upload/page/0091/WU%2...,Microsoft Word - WU District of Innovation.doc...,"[25.081, 21.401, 21.003, 25.0811, 25.036]"
17,17,Waxahachie ISD,https://1.cdn.edl.io/D035lUeGyDjps8WLLMFO9oWnG...,Seeking District of Innovation Designation: Cr...,"[25.082, 25.081, 21.352, 25.0811, 25.083]"


In [53]:
missing_text = missing_links[missing_links['possible_laws'].astype(str) == '[]']
missing_text.head()

,Unnamed: 0,title,link,text,possible_laws
3,3,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,No text,[]
6,6,Wink-Loving ISD,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,Wink-Loving ISD WLISD - The Best in Texas: Bui...,[]
7,7,Windthorst ISD,http://www.windthorstisd.net/Uploads/50/misc/f...,UNAVAILABLE,[]
9,9,Willis ISD,https://tx50000123.schoolwires.net//cms/lib/TX...,"2 Approved by WISD School Board April 12, 2017...",[]
10,10,Whitney ISD,https://www.whitney.k12.tx.us/cms/lib3/TX01001...,No text,[]


In [54]:
len(missing_text) #was129

149

In [55]:
print('We have added', len(missing_links_corrected), 'districts.')
print(len(missing_text), 'are still missing.') 

We have added 127 districts.
149 are still missing.


## Address districts that are missing due to missing or incorrect text.

### Where possible, incorporate previously (manually) collected text before editing

In [56]:
# save as new
missing_text_new = missing_text.rename({'text': 'text_scraped'}, axis = 'columns')

# import old
missing_text_old = pd.read_csv(os.path.join(data_path, 'missing_text_corrected.csv'), encoding='latin-1')
missing_text_old = missing_text_old.rename({'text': 'text_manual'}, axis = 'columns')

# merge new and old
missing_text = missing_text_new[['title', 'link', 'text_scraped']].merge(missing_text_old[['title', 'text_manual']], how = 'left', left_on = 'title', right_on = 'title')
print("Number of missing texts", len(missing_text))

# set text equal to text manual
missing_text['text'] = missing_text.text_manual

# replace text to text scraped if text is missing
missing_text['text'] = missing_text['text'].fillna(missing_text['text_scraped'])

missing_text = missing_text[['title', 'link', 'text']]
missing_text.to_csv(os.path.join(data_path, 'missing_text.csv'))

Number of missing texts 149


##### Manually add correct text. If no text (photocopy), put 'No text' in cell. If you cannot find the document, put 'No document found'. Save as missing_text_corrected.csv.

In [57]:
missing_texts = pd.read_csv(os.path.join(data_path, 'missing_text_corrected.csv'))
missing_texts = clean_documents.remove_whitespace(missing_texts, 'text')
missing_texts.head(20)

,Unnamed: 0,title,link,text
0,0,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,No text
1,1,Wink-Loving ISD,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,Wink-Loving ISD WLISD - The Best in Texas: Bui...
2,2,Windthorst ISD,http://www.windthorstisd.net/Uploads/50/misc/f...,No text
3,3,Willis ISD,https://tx50000123.schoolwires.net//cms/lib/TX...,"2 Approved by WISD School Board April 12, 2017..."
4,4,Whitney ISD,https://www.whitney.k12.tx.us/cms/lib3/TX01001...,No text
5,5,Whitharral ISD,https://docs.google.com/document/d/1QpBqSAmJNZ...,Whitharral ISD District of Innovation  Comm...
6,6,White Oak ISD,https://docs.google.com/document/d/1otlNz4M2pp...,A DISTRICT PLAN FOR INNOVATION & LOCAL CONTROL...
7,7,West Sabine ISD,http://www.westsabineisd.net/page/open/921/0/2...,No text
8,8,Waco ISD,https://docs.google.com/viewerng/viewer?url=ht...,WACO INDEPENDENT SCHOOL DISTRICT DISTRICT OF I...
9,9,Valley View ISD,https://s3.amazonaws.com/scschoolfiles/890/dis...,1 SUBCHAPTER F. DISTRICT-LEVEL AND SITE-BASED ...


### Extract laws

In [58]:
missing_texts['possible_laws'] = missing_texts.text.apply(extract_laws.get_laws)

In [59]:
missing_texts.head(20)

,Unnamed: 0,title,link,text,possible_laws
0,0,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,No text,[]
1,1,Wink-Loving ISD,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,Wink-Loving ISD WLISD - The Best in Texas: Bui...,[]
2,2,Windthorst ISD,http://www.windthorstisd.net/Uploads/50/misc/f...,No text,[]
3,3,Willis ISD,https://tx50000123.schoolwires.net//cms/lib/TX...,"2 Approved by WISD School Board April 12, 2017...",[]
4,4,Whitney ISD,https://www.whitney.k12.tx.us/cms/lib3/TX01001...,No text,[]
5,5,Whitharral ISD,https://docs.google.com/document/d/1QpBqSAmJNZ...,Whitharral ISD District of Innovation  Comm...,"[37.008, 25.081, 25.036, 21.003, 21.3541, 21.2..."
6,6,White Oak ISD,https://docs.google.com/document/d/1otlNz4M2pp...,A DISTRICT PLAN FOR INNOVATION & LOCAL CONTROL...,"[37.0012, 25.0811, 25.082, 21.003, 22.011, 25...."
7,7,West Sabine ISD,http://www.westsabineisd.net/page/open/921/0/2...,No text,[]
8,8,Waco ISD,https://docs.google.com/viewerng/viewer?url=ht...,WACO INDEPENDENT SCHOOL DISTRICT DISTRICT OF I...,"[25.094, 15.113, 21.003, 21.044, 21.102, 21.05..."
9,9,Valley View ISD,https://s3.amazonaws.com/scschoolfiles/890/dis...,1 SUBCHAPTER F. DISTRICT-LEVEL AND SITE-BASED ...,[]


In [60]:
missing_text_corrected = missing_texts[missing_texts['possible_laws'].astype(str) != '[]']
missing_text_corrected

,Unnamed: 0,title,link,text,possible_laws
5,5,Whitharral ISD,https://docs.google.com/document/d/1QpBqSAmJNZ...,Whitharral ISD District of Innovation  Comm...,"[37.008, 25.081, 25.036, 21.003, 21.3541, 21.2..."
6,6,White Oak ISD,https://docs.google.com/document/d/1otlNz4M2pp...,A DISTRICT PLAN FOR INNOVATION & LOCAL CONTROL...,"[37.0012, 25.0811, 25.082, 21.003, 22.011, 25...."
8,8,Waco ISD,https://docs.google.com/viewerng/viewer?url=ht...,WACO INDEPENDENT SCHOOL DISTRICT DISTRICT OF I...,"[25.094, 15.113, 21.003, 21.044, 21.102, 21.05..."
10,10,Valentine ISD,http://www.valentineisd.com/search?orgId=14e1e...,Valentine ISD District of Innovation Plan Augu...,"[25.081, 21.354, 21.003, 21.353, 21.352, 25.08..."
13,13,Trenton ISD,https://docs.google.com/document/d/1RpjiOsZ-rv...,Trenton ISD District of Innovation Plan (HB 18...,"[25.111, 21.102, 21.003, 25.0811, 25.113, 25.112]"
14,14,Tornillo ISD,http://www.tisd.us/district_accountability/dis...,2018 - 2023 District of Innovation Plan Tornil...,"[37.002, 37.0012, 28.0216, 25.081, 25.1113, 21..."
15,15,Tidehaven ISD,https://drive.google.com/drive/folders/0B-ALJg...,District of Innovation Plan Tidehaven ISD Augu...,"[25.081, 25.036, 21.401, 21.003, 21.057, 21.10..."
18,18,Tenaha ISD,https://docs.google.com/viewerng/viewer?url=ht...,TENAHA ISD DISTRICT OF INNOVATION PLAN (HB 184...,"[11.255, 11.251, 11.253, 11.252, 21.003, 21.40..."
24,24,Spearman ISD,https://docs.google.com/document/d/1hmmYyKFbhv...,"Introduction House Bill (HB) 1842, passe...","[37.008, 25.082, 25.0811, 11.251, 21.401, 21.3..."
25,25,Slocum ISD,https://docs.google.com/viewer?a=v&pid=sites&s...,Slocum ISD District of Innovation Plan (HB 184...,"[25.113, 25.0811, 21.102, 21.3541, 21.352, 21...."


In [61]:
missing_laws = missing_texts[missing_texts['possible_laws'].astype(str) == '[]']
print('We have added', len(missing_text_corrected), 'districts.')
print(len(missing_laws), 'are still missing.') # 89 still missing

We have added 41 districts.
106 are still missing.


# Address districts that are photocopies

#### Incorporate previous edits

In [62]:
# Import old manual edits
missing_laws_old = pd.read_csv(os.path.join(data_path, 'missing_laws_corrected.csv'), encoding='latin-1')
missing_laws_old = missing_laws_old.rename({'possible_laws': 'laws_old'}, axis = 'columns')

# Merge with new missing obs
missing_laws = missing_laws.merge(missing_laws_old[['title',  'laws_old']], how = 'left', left_on = 'title', right_on = 'title')
print("Number of missing laws", len(missing_laws))

missing_laws['possible_laws'] = missing_laws.laws_old
missing_laws = missing_laws[['title', 'possible_laws','link']]
missing_laws.to_csv(os.path.join(data_path, 'missing_laws.csv'))
missing_laws.head()

Number of missing laws 106


,title,possible_laws,link
0,Woodville ISD,"[25.0811, 25.0812]",https://core-docs.s3.amazonaws.com/documents/a...
1,Wink-Loving ISD,"[25.0811, 21.003]",https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...
2,Windthorst ISD,"[21.401, 25.0811]",http://www.windthorstisd.net/Uploads/50/misc/f...
3,Willis ISD,"[25.0811, 21.003, 21.102]",https://tx50000123.schoolwires.net//cms/lib/TX...
4,Whitney ISD,[25.0811],https://www.whitney.k12.tx.us/cms/lib3/TX01001...


In [63]:
missing_laws

,title,possible_laws,link
0,Woodville ISD,"[25.0811, 25.0812]",https://core-docs.s3.amazonaws.com/documents/a...
1,Wink-Loving ISD,"[25.0811, 21.003]",https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...
2,Windthorst ISD,"[21.401, 25.0811]",http://www.windthorstisd.net/Uploads/50/misc/f...
3,Willis ISD,"[25.0811, 21.003, 21.102]",https://tx50000123.schoolwires.net//cms/lib/TX...
4,Whitney ISD,[25.0811],https://www.whitney.k12.tx.us/cms/lib3/TX01001...
5,West Sabine ISD,"[21.003, 21.0031, 25.0811, 25.0812, 21.401, 21...",http://www.westsabineisd.net/page/open/921/0/2...
6,Valley View ISD,"11.251, 11.252, 11.253, 21.003, 21.102, 21.401...",https://s3.amazonaws.com/scschoolfiles/890/dis...
7,Troup ISD,"[25.0811, 21.102]",https://www.troupisd.org/1617/innovation.pdf
8,Trinidad ISD,"[25.0811, 21.003, 21.057, 25.081, 21.102]",http://www.trinidadisd.com/UserFiles/Servers/S...
9,Thorndale ISD,"[21.003, 21.053, 21.057, 21.102, 21.352, 21.35...",http://www.thorndale.txed.net/files/user/21/fi...


##### Manually add correct laws. If you cannot find the document, leave possible_laws blank. Save as missing_laws_corrected.csv.

In [64]:
missing_laws_corrected = pd.read_csv(os.path.join(data_path, 'missing_laws_corrected.csv'))
missing_laws_corrected.head(5)

,Unnamed: 0,title,possible_laws,link
0,0,Woodville ISD,"[25.0811, 25.0812]",https://core-docs.s3.amazonaws.com/documents/a...
1,1,Wink-Loving ISD,"[25.0811, 21.003]",https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...
2,2,Windthorst ISD,"[21.401, 25.0811]",http://www.windthorstisd.net/Uploads/50/misc/f...
3,3,Willis ISD,"[25.0811, 21.003, 21.102]",https://tx50000123.schoolwires.net//cms/lib/TX...
4,4,Whitney ISD,[25.0811],https://www.whitney.k12.tx.us/cms/lib3/TX01001...


# Update full dataset

In [65]:
missing_links_corrected = missing_links_corrected.set_index('title')
missing_text_corrected = missing_text_corrected.set_index('title')
missing_laws_corrected = missing_laws_corrected.set_index('title')

In [66]:
docs_df = docs_df.set_index('title')
docs_df = docs_df.sort_index()
docs_df.update(missing_links_corrected)
docs_df.update(missing_text_corrected)
docs_df.update(missing_laws_corrected)
docs_df['possible_laws'] = np.where((docs_df.possible_laws is None), '[]', docs_df.possible_laws)
docs_df.sample(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,level,type,link,text,p_innovation,possible_laws
title,,,,,,,,,,,
Kerens ISD,159.0,261,261,261,261,First,pdf,https://s3.amazonaws.com/scschoolfiles/585/ker...,Kerens ISD District of Innovation Plan Introdu...,0.000045,"[37.0012, 25.082, 25.111, 25.081, 21.354, 21.0..."
Bowie ISD,97.0,37,37,37,37,First,pdf,http://bowie.esc11.net//cms/lib6/TX02218883/Ce...,Not a plan,0.000045,"[25.0811, 25.082, 21.102, 37.0012, 25.092, 21...."
Brackett ISD,747.0,332,332,332,332,First,pdf,http://www.brackettisd.net/required_postings/d...,Brackett Independent School District District ...,0.999955,"[25.082, 21.003, 25.081, 25.0811]"
Deer Park ISD,621.0,2326,2326,2326,2582,Second,pdf,http://www.dpisd.org/cms/lib/TX01001079/Centri...,Microsoft Word - District of Innovation Plan -...,0.998653,"[25.0811, 21.352, 37.0012, 25.082, 25.092]"
Jefferson ISD,166.0,207,207,207,207,First,pdf,https://s3.amazonaws.com/scschoolfiles/1488/di...,Microsoft Word - JISD District of Innovation P...,0.000045,"[37.0012, 44.902, 21.057, 21.102, 21.003, 25.0..."
Lometa ISD,140.0,19,19,19,19,First,google,https://drive.google.com/file/d/1DqPY5G88CnmpB...,LOMETA INDEPENDENT SCHOOL DISTRICT Distric...,0.999955,"[25.114, 25.111, 25.081, 21.102, 21.354, 21.00..."
Community ISD,656.0,418,418,418,418,First,pdf,http://www.communityisd.org/site/handlers/file...,Microsoft Word - DOI-PLAN-REVISED-4-12-17 (1)....,0.999955,"[37.0012, 21.003, 44.902, 25.036, 21.102, 21.4..."
Paradise ISD,253.0,336,336,336,336,First,pdf,http://www.pisd.net/PISD/Admin/Info/doi.pdf,Microsoft Word - Paradise ISD DOI 2017 as of 3...,0.999955,"[37.0012, 25.0811, 45.209, 45.205, 45.206, 25...."
Collinsville ISD,664.0,1934,1934,1934,2129,Second,pdf,https://www.collinsvilleisd.org/cms/lib6/TX018...,Microsoft Word - Collinsville ISD District of ...,0.998273,"[25.0811, 21.003, 21.102, 21.401, 25.112, 25.113]"


In [67]:
docs_df[docs_df.index == "Liberty Hill ISD"]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,level,type,link,text,p_innovation,possible_laws
title,,,,,,,,,,,
Liberty Hill ISD,384.0,3901,3901,3901,302,html,html,https://www.libertyhill.txed.net/Domain/294,Not a plan,0.092454,[]


In [68]:
for law in docs_df[docs_df.index == "Blum ISD"].possible_laws:
    print(law)

[11.251, 21.352, 21.451, 21.458, 21.003, 25.113, 25.082, 25.0811, 25.092, 25.087, 25.083, 25.084, 25.036, 25.081, 28.0216, 29.0821, 28.0214, 37.0012, 45.206, 21.401, 21.102, 21.353, 21.3541, 21.354, 25.112]


In [69]:
print("Only missing", len(docs_df[docs_df['possible_laws'].astype(str) == '[]']), " of ", len(docs_df),  "- unable to find plans.")

Only missing 10  of  824 - unable to find plans.


# Save

In [70]:
docs_df.to_csv(os.path.join(data_path, 'doi_exemptions_list.csv'))